In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import re
import numpy as np
from bs4 import BeautifulSoup as bs
import pickle
import pandas as pd

import csv

import warnings
warnings.filterwarnings('ignore')

In [29]:
cardigan=pd.read_csv('C:/Temp/크롤링/가디건/cardigan.csv')
shirts=pd.read_csv('C:/Temp/크롤링/셔츠/shirts.csv')
blouses=pd.read_csv('C:/Temp/크롤링/블라우스/blouses.csv')
pullover=pd.read_csv('C:/Temp/크롤링/니트/pullover.csv')
long_sleeve=pd.read_csv('C:/Temp/크롤링/긴팔티/long_sleeve.csv')
short_sleeve=pd.read_csv('C:/Temp/크롤링/반팔티/short_sleeve.csv')
jackets=pd.read_csv('C:/Temp/크롤링/자켓/jackets.csv')
leather_jackets=pd.read_csv('C:/Temp/크롤링/가죽자켓/leather_jackets.csv')
coat=pd.read_csv('C:/Temp/크롤링/코트/coat.csv')
jumper=pd.read_csv('C:/Temp/크롤링/점퍼/jumper.csv')
# 17056개

long_midi=pd.read_csv('C:/Temp/크롤링/롱&미디 데님/long_midi.csv')
short=pd.read_csv('C:/Temp/크롤링/쇼트 데님/short.csv')
long_midi_formal=pd.read_csv('C:/Temp/크롤링/롱&미디 포멀/long_midi_formal.csv')
short_formal=pd.read_csv('C:/Temp/크롤링/쇼트 포멀/short_formal.csv')
long_midi_skirt=pd.read_csv('C:/Temp/크롤링/롱&미디 스커트/long_midi_skirt.csv')
short_skirt=pd.read_csv('C:/Temp/크롤링/쇼트 스커트/short_skirt.csv')
# 3971개


# 총 21027

In [7]:
clothes=pd.concat([cardigan,shirts,blouses,pullover,long_sleeve,short_sleeve,jackets,leather_jackets,coat,jumper,long_midi,short,long_midi_formal,short_formal,long_midi_skirt,short_skirt])
len(clothes)
clothes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21027 entries, 0 to 348
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   img     21027 non-null  object
 1   like    15406 non-null  object
 2   color   21027 non-null  object
 3   type    21027 non-null  object
dtypes: object(4)
memory usage: 821.4+ KB


In [18]:
idx=[]
for i in range(1,21028):
    idx.append(i)
len(idx)

21027

In [22]:
clothes['index']=idx
clothes.reset_index(inplace=True)
clothes.drop('level_0',axis=1,inplace=True)
clothes

,img,like,color,type,index
0,https://img.ssfshop.com/cmd/LB_500x660/src/htt...,81,검정색,가디건,1
1,https://img.ssfshop.com/cmd/LB_500x660/src/htt...,146,검정색,가디건,2
2,https://img.ssfshop.com/cmd/LB_500x660/src/htt...,93,검정색,가디건,3
3,https://img.ssfshop.com/cmd/LB_500x660/src/htt...,102,검정색,가디건,4
4,https://img.ssfshop.com/cmd/LB_500x660/src/htt...,45,검정색,가디건,5
...,...,...,...,...,...
21022,https://img.ssfshop.com/cmd/LB_500x660/src/htt...,NaN,갈색,쇼트 스커트,21023
21023,https://img.ssfshop.com/cmd/LB_500x660/src/htt...,2,갈색,쇼트 스커트,21024
21024,https://img.ssfshop.com/cmd/LB_500x660/src/htt...,NaN,갈색,쇼트 스커트,21025
21025,https://img.ssfshop.com/cmd/LB_500x660/src/htt...,3,갈색,쇼트 스커트,21026


In [15]:
count=clothes.groupby(['type','color']).size()
count.sort_values(ascending=False).reset_index()

,type,color,0
0,블라우스,흰색,584
1,니트,흰색,538
2,롱&미디 데님,파란색,497
3,자켓,검정색,492
4,롱&미디 스커트,검정색,482
...,...,...,...
128,롱&미디 포멀,보라색,10
129,쇼트 포멀,회색,9
130,가죽자켓,분홍색,4
131,쇼트 포멀,갈색,3


In [28]:
clothes[['type','color','like','img']].groupby(['type','color']).max('like')

img
type color                                                   
가디건  갈색     https://img.ssfshop.com/cmd/LB_500x660/src/htt...
     검정색    https://img.ssfshop.com/cmd/LB_500x660/src/htt...
     노란색    https://img.ssfshop.com/cmd/LB_500x660/src/htt...
     보라색    https://img.ssfshop.com/cmd/LB_500x660/src/htt...
     분홍색    https://img.ssfshop.com/cmd/LB_500x660/src/htt...
...                                                       ...
코트   빨간색    https://img.ssfshop.com/cmd/LB_500x660/src/htt...
     초록색    https://img.ssfshop.com/cmd/LB_500x660/src/htt...
     파란색    https://img.ssfshop.com/cmd/LB_500x660/src/htt...
     회색     https://img.ssfshop.com/cmd/LB_500x660/src/htt...
     흰색     https://img.ssfshop.com/cmd/LB_500x660/src/htt...

[133 rows x 1 columns]

In [39]:
color_img=clothes[['color']]
color_img.head()

,color
0,검정색
1,검정색
2,검정색
3,검정색
4,검정색


In [40]:
idx=[]
for i in range(1,21028):
    idx.append(i)
len(idx)

21027

In [41]:
color_img.index=idx
color_img.reset_index(inplace=True)

In [47]:
# color_img
color_img.to_csv('c:/Temp/크롤링/8seconds/color.csv',encoding='utf-8-sig',index=False)


,index,color
0,1,검정색
1,2,검정색
2,3,검정색
3,4,검정색
4,5,검정색
...,...,...
21022,21023,갈색
21023,21024,갈색
21024,21025,갈색
21025,21026,갈색


In [25]:
color_img['img'].isnull().sum()

0

In [20]:
img_list=list(color_img['img'])

In [31]:
from urllib.request import urlopen
# from requests.packages.urllib3.exceptions import InsecureRequestWarning
# requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
n=12684
for img_src in img_list[12683:]:
    with urlopen(img_src) as f:
        # img라는 폴더 안에 programmer(n).jpg 파일을 저장한다.
        # 텍스트 파일이 아니기 때문에 w(write)만 쓰면 안되고 binary 모드를 추가시켜야 한다.
        with open('c:/Temp/크롤링/8seconds/' + str(n) + '.jpg', "wb") as h:
            # f를 읽고 img에 저장한다.
            img = f.read()
            # h에 위 내용을 쓴다.
            h.write(img)
    # 계속 programmer 1에 덮어쓰지 않도록 1을 증가시켜 준다
    n += 1 

In [ ]:
21027
21028

In [34]:
clothes[clothes['img']=='https://img.ssfshop.com/cmd/LB_500x660/src/https://img.ssfshop.com/goods/ORBR/20/03/12/GPWY20031292071_1_ORGINL_20200312140935816.jpg']

,img,like,color,type,index
3348,https://img.ssfshop.com/cmd/LB_500x660/src/htt...,4,흰색,블라우스,3349


In [ ]:
Deux Fois H 
MR. by Lang
